# **Installations:**

In [2]:
!pip install -q nltk pycocoevalcap
!pip install -q nltk pandas
!pip install -q sacrebleu
!pip install -q bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 826.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 982.3 kB/s eta 0:00:00


In [3]:
import sacrebleu
import pandas as pd
import ast
import nltk
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from bert_score import score

### Data preparation:

In [65]:
# strange_df = pd.read_csv("strange_answers.csv")
# # Function to extract the embedded answer
# def extract_embedded_answer(question):
#     try:
#         # Convert string representation of list to actual list
#         question_list = ast.literal_eval(question)
#         # print(question_list)
#         if isinstance(question_list, list) and len(question_list) > 1:
#             return question_list[1]
#         else:
#             raise ValueError("Question does not have the expected format.")
#     except (ValueError, SyntaxError) as e:
#         print(f"Error parsing question: {question} - {e}")
#         return None

# # Apply function to extract the embedded answer
# strange_df['gold answers'] = strange_df['question'].apply(extract_embedded_answer)

# # Filter out rows where parsing failed
# strange_df = strange_df.dropna(subset=['gold answers'])

In [66]:
# strange_df['gold answers']

,gold answers
0,plowing sand
1,bottom
2,rough waters
3,blackboard
4,santa
...,...
96,playing
97,The tiger staring at zebras
98,Elsa
99,his left wrist


In [71]:
# normal_df = pd.read_csv("normal question.csv")
# normal_df['normal gold answers'].to_csv("gold_normal_questions_normal_images.csv", index=False)
# normal_df['weird gold answers'].to_csv("gold_normal_questions_weird_images.csv", index=False)

In [67]:
# strange_df['gold answers'].to_csv("gold_weird_questions_weird_images.csv")

In [83]:
op1 = "gold_weird_questions_weird_images.csv"
op2 = "gold_normal_questions_weird_images.csv"
op3 = "gold_normal_questions_normal_images.csv"

gold_df = pd.read_csv(op2)

input_file_for_calculation.csv is not a real file, it can be any file you want to check it's scores

In [80]:
input_file_for_calculation = "strange_answers_gpt_prompt_no explain_blip.csv"

In [84]:
# Ensure you have the necessary NLTK data files
nltk.download('punkt')

# Load data from CSV
df = pd.read_csv(input_file_for_calculation)

# Function to extract the embedded answer
def extract_embedded_answer(question):
    try:
        # Convert string representation of list to actual list
        question_list = ast.literal_eval(question)
        # print(question_list)
        if isinstance(question_list, list) and len(question_list) > 1:
            return question_list[1]
        else:
            raise ValueError("Question does not have the expected format.")
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing question: {question} - {e}")
        return None

# Apply function to extract the embedded answer
# df['embedded_answer'] = df['question'].apply(extract_embedded_answer)
df['embedded_answer'] = gold_df['gold answers']
# Filter out rows where parsing failed
df = df.dropna(subset=['embedded_answer'])

# Prepare the data for evaluation
gts = {idx: [row['answer']] for idx, row in df.iterrows()}
res = {idx: [row['embedded_answer']] for idx, row in df.iterrows()}
references =df['answer'].tolist()
candidates = df['embedded_answer'].tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:
#Checking the expected data
for i in candidates:
  print(i)

print(len(references))
print(len(candidates))

yellow
Circular
wool
equasion
red and white suit
brown
pink
the bride
sand
brown
white
ancient egypt
three
4
white
brown
glass
helmet with flashlifht
right
5
a bandana
brown
red
steam train
brown
jeans , sneakers and t-shirt
white
red
white
twigs
in the desert
Vintage curls
In a cave.
brown
Serious
orange
a wall
Round
wood
gray
4
nature and trees
blue shirt
orange
tank top
3
6
green
2
gray
Teal.
S
many
8
green
1
black
right
black
red
orange and silver
1
grass
hat
8
raining
10:10
white
orange
cone
blue and gold
blue and gray
black and gold
white
in the street
Floral crown
green
star of david
orange
2
brown and green
1
gold
white
2
red
black
brown
black
2
black and white
white
4
2
in the desert
blue
curly
4
dalannas
2
black golf
101
101


### Calculate CIDEr score:

In [50]:
# Initialize the evaluators
scorers = [
    (Cider(), "CIDEr"),
]

all_results_dict = {}

# Calculate the scores
for scorer, method in scorers:
    print(f'Computing {method} score...')
    score, scores = scorer.compute_score(gts, res)
    if isinstance(score, list):
        all_results_dict[method] = round(score[3] * 100, 2)
        print(f"{method}: {round(score[3] * 100, 2)}\n")
    else:
        all_results_dict[method] = round(score * 100, 2)
        print(f"{method}: {round(score * 100, 2)}\n")

# Print the overall results
print(all_results_dict)

Computing CIDEr score...


AttributeError: 'float' object has no attribute 'split'

In [41]:
import math


### Calculate SacreBLEU score:

In [86]:
bleu_scores = []

# Calculate SacreBLEU score for each pair
for i in range(len(references)):
    if references[i] is None or hypothesis is None or isinstance(references[i], float) and math.isnan(references[i]):
      bleu_scores.append(0.0)
    else:
      # sacrebleu expects a list of references
      reference = [references[i]]
      hypothesis = candidates[i]

      bleu = sacrebleu.sentence_bleu(hypothesis, reference)
      bleu_scores.append(bleu.score)

sumblue = 0
for i in bleu_scores:
  sumblue += float(i)

# Calculate the average BLEU score
average_bleu = sumblue / len(bleu_scores)
print(f"Average SacreBLEU score: {average_bleu:.2f}")

Average SacreBLEU score: 0.00


## Calculate Bert score:

### Calculate bert score with a simple example:

In [16]:
# Define the reference and candidate sentences
references = ["ocean"]
candidates = ["sea"]

# Calculate BERTScore
P, R, F1 = score(candidates, references, lang="en", verbose=True)

# Print the scores
print(f"Precision: {P.mean().item():.4f}")
print(f"Recall: {R.mean().item():.4f}")
print(f"F1: {F1.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.28 seconds, 3.53 sentences/sec
Precision: 0.9980
Recall: 0.9980
F1: 0.9980


### Calculate bert score for the given file:

In [87]:
from ast import literal_eval  # To safely evaluate the string representation of lists

# Compute BERTScore for each row
bert_scores = []
sum_bert = 0
for index, row in df.iterrows():
    actual_answer = row['answer']
    embedded_answer = row['embedded_answer']

    if isinstance(actual_answer, float) and math.isnan(actual_answer):
      bert_scores.append(0.0)  # Assign the lowest score (0.0)
      sum_bert += 0.0
    else:
      P, R, F1 = score([actual_answer], [embedded_answer], lang='en', model_type='bert-base-uncased')
      bert_scores.append(F1.item())
      sum_bert += F1.item()

average_bert_score = sum_bert / len(bert_scores)
print(f"Average BERTScore: {average_bert_score:.4f}")




Average BERTScore: 0.0040
